# 五 文件和进程

- 文本文件读写
- json和csv文件操作
- 序列化及应用
- 多进程与多线程
- 进程池和线程池
- 系统库

## 文件

In [6]:
f = open("sample.txt","r")#r = read ,w = write, a = append , b = binary 
print(type(f))
texts = f.read()
print(texts)
f.close()

<class '_io.TextIOWrapper'>
Line #1 hello world
Line #2 life is not easy


In [7]:
with open("sample.txt","r") as f:
    line = f.readline()
    while line:
        print(line.strip())
        line = f.readline()

Line #1 hello world
Line #2 life is not easy


In [8]:
with open("sample.txt","r") as f:
    for line in f.readlines():
        print(line.strip())

Line #1 hello world
Line #2 life is not easy


In [9]:
#实现readline 生成器
def readlines(f):
    line = f.readline()
    while line:
        yield line
        line = f.readline()  
        
with open("sample.txt","r") as f:
    for line in f.readlines():
        print(line.strip())

Line #1 hello world
Line #2 life is not easy


In [11]:
# 写文件
texts = ['New Line #1 hello world', 'Line #2 life is not easy']

with open("new_sample.txt","w+") as f:
    for text in texts:
        f.write(text + '\n')
        
with open("new_sample.txt", 'a+') as f:
    f.write('Something new \n')

In [17]:
#json 文件的读取

"""
json 序列化和反序列化
#序列化：  dump()/dumps():将python中的字典的对象序列化后转换成json格式的字符串
#反序列化：load()/loads():将json格式字符串转化成pyhon中的字典对象
""" 
# dump（）：将python中的字典的对象序列化转换成json格式文件并保存到json文件中
# dumps（）：将python中的字典的对象序列化转换成json格式的字符串对象并保存到json文件中
# loads（）：将json格式字符串转化成pyhon中的字典对象

import json
# json.dump 和json.load
# 序列化操作
config = {'ip':'192.168.1.1', 'port':['9100', '9101', '9102']}
with open('config.json', 'w+') as f:
    json.dump(config, f) #把字典写入json文件，传入了文件句柄
# 反序列化操作
with open('config.json', 'r') as f:
    new_config = json.load(f) #把json文件对象变成字典对象
print(type(new_config))
print(new_config)

# json.dumps 和json.loads
config_str = '{"ip": "192.168.1.1", "port": ["9100", "9101", "9102"]}'#json中字符串用“”双引号
config = json.loads(config_str)
print(config)
new_config_str = json.dumps(config)
print(type(new_config_str))
print(new_config_str)

<class 'dict'>
{'ip': '192.168.1.1', 'port': ['9100', '9101', '9102']}
{'ip': '192.168.1.1', 'port': ['9100', '9101', '9102']}
<class 'str'>
{"ip": "192.168.1.1", "port": ["9100", "9101", "9102"]}


In [19]:
#模拟dumps的实现：把字典对象序列化成json对象
def json_dumps(di):
    lines = []
    s = '{\n'
    for k, v in di.items():
        _s = '"' + k + '":'
        if type(v) != list:
            _s += '"' + v + '"'
        else:
            items = ['"' + i + '"' for i in v ]
            _s += '[' + ','.join(items)  + ']'
        lines.append(_s)
    s += ',\n'.join(lines)
    s +='\n}'
    return s

config = {'ip':'192.168.1.1', 'port':['9100', '9101', '9102']}
print(json_dumps(config))

{
"ip":"192.168.1.1",
"port":["9100","9101","9102"]
}


In [31]:
#序列化和反序列化 pickle
#按对象进行输出和输入
import pickle

class MyObject:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
obj = MyObject(100, 200)
s_obj = pickle.dumps(obj)
print(s_obj)
obj = pickle.loads(s_obj)
print(obj.x, obj.y)



b'\x80\x03c__main__\nMyObject\nq\x00)\x81q\x01}q\x02(X\x01\x00\x00\x00xq\x03KdX\x01\x00\x00\x00yq\x04K\xc8ub.'
100 200


In [20]:
#csv 读取文件
import csv

feature_list = []
label_list = []

with open("sales.csv", 'r') as f:
    reader = csv.reader(f)
    header = next(reader)# 第一行是标题行
    for row in reader:   #从第二行开始读取每条记录属性
        label_list.append(row[-1]) 
        row_dict = {}
        for i in range(1, len(row) -1):
            row_dict[header[i]] = row[i]
        feature_list.append(row_dict)
        
print(feature_list)
print(label_list)

[{'Age': 'youth', 'Income': 'high', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'youth', 'Income': 'high', 'Student': 'no', 'CreditRating': 'excellent'}, {'Age': 'middle', 'Income': 'high', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'senior', 'Income': 'medium', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'senior', 'Income': 'low', 'Student': 'yes', 'CreditRating': 'fair'}, {'Age': 'senior', 'Income': 'low', 'Student': 'yes', 'CreditRating': 'excellent'}, {'Age': 'middle', 'Income': 'low', 'Student': 'yes', 'CreditRating': 'excellent'}, {'Age': 'youth', 'Income': 'medium', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'youth', 'Income': 'low', 'Student': 'yes', 'CreditRating': 'fair'}, {'Age': 'senior', 'Income': 'medium', 'Student': 'yes', 'CreditRating': 'fair'}, {'Age': 'youth', 'Income': 'medium', 'Student': 'yes', 'CreditRating': 'excellent'}, {'Age': 'middle', 'Income': 'medium', 'Student': 'no', 'CreditRating': 'excellent'}, {'Age': 'middle', 'Income': 'hi

In [27]:
#实现csv文件的读取
def read_csv(f):
    line = f.readline()
    while line:
        line = line.strip()
        yield line.split(',')#按逗号分隔
        line = f.readline()

headers = None
feature_list = []
label_list = []

with open("sales.csv", 'r') as f:
    for row in read_csv(f):   #从第二行开始读取每条记录属性
        print(row)
        if not headers:#第一行标题行
            headers = row
        else:
            label_list.append(row[-1]) 
            row_dict = {}
            for i in range(1, len(row) -1):
                row_dict[headers[i]] = row[i]
            feature_list.append(row_dict)
        
print(feature_list)
print(label_list)

['RID', 'Age', 'Income', 'Student', 'CreditRating', 'BuyComputer']
['1', 'youth', 'high', 'no', 'fair', 'no']
['2', 'youth', 'high', 'no', 'excellent', 'no']
['3', 'middle', 'high', 'no', 'fair', 'yes']
['4', 'senior', 'medium', 'no', 'fair', 'yes']
['5', 'senior', 'low', 'yes', 'fair', 'yes']
['6', 'senior', 'low', 'yes', 'excellent', 'no']
['7', 'middle', 'low', 'yes', 'excellent', 'yes']
['8', 'youth', 'medium', 'no', 'fair', 'no']
['9', 'youth', 'low', 'yes', 'fair', 'yes']
['10', 'senior', 'medium', 'yes', 'fair', 'yes']
['11', 'youth', 'medium', 'yes', 'excellent', 'yes']
['12', 'middle', 'medium', 'no', 'excellent', 'yes']
['13', 'middle', 'high', 'yes', 'fair', 'yes']
['14', 'senior', 'medium', 'no', 'excellent', 'no']
[{'Age': 'youth', 'Income': 'high', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'youth', 'Income': 'high', 'Student': 'no', 'CreditRating': 'excellent'}, {'Age': 'middle', 'Income': 'high', 'Student': 'no', 'CreditRating': 'fair'}, {'Age': 'senior', 'Income

## 多进程和多线程

- 进程：是计算机中的程序关于其中所需要的的数据集合的活动，是os结构的基本单位
- 线程：轻量级的进程，同一个进程中包含多个线程

In [2]:
#多进程: 代码最好在cmd中运行，因为子进程的id在note中看不到
from multiprocessing import Process
import os

#子进程
def run_pro(name):
    print("Run child process %s (%s)..." %(name, os.getpid()))

if __name__ == '__main__':
    print("Parent process %s." % os.getpid())
    p = Process(target = run_pro, args = ('test', ))
    print('Child process will start .')
    p.start()
    p.join()# 等待进程或线程结束
    print('Child process end')

Parent process 8420.
Child process will start .
Child process end


In [4]:
#多线程：python中是没有线程id的概念，但有名字
import time, threading

#新线程
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n <5:
        n = n+1
        print("thread %s >>> %s" %(threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target =loop, name="LoopThread")
t.start()
t.join()
print('thread %s ended . ' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended
thread MainThread ended . 


In [ ]:
#进程池：进程多了会导致上下文切换的问题=》代码管理成本增加
#注意：此cell代码在cmd中运行
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print("Run task %s (%s)..." %(name, os.getpid))
    start = time.time()
    time.sleep(random.random()*3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' %(name, (end - start)))

if __name__ == "__main__":
    print("Parent process %s." % os.getpid())
    p = Pool(4) #进程池里同时最多跑的进程个数，一般设置为核的两倍
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()#close:不再往进程池添加任务
    p.join()
    print("All subprocesses done.")
    

Parent process 8420.
Waiting for all subprocesses done...


In [2]:
#线程池
#步骤：
#1. 定义线程函数
#2. 将创建多个任务put到线程池中--threadpool.makeRequests()
#3. 等待所有任务处理完毕 ---threadpool.wait
import threadpool
import os, time, random

def long_op(x):
    print('%d \\n' %x)
    time.sleep(2)
pool = threadpool.ThreadPool(os.cpu_count())
tasks = threadpool.makeRequests(long_op, [1,2,3,4,5,6,7,8,9,10])
print(len(tasks))
[pool.putRequest(task)  for task in tasks]
pool.wait()

10
1 \n
3 \n
2 \n4 \n

5 \n
6 \n
7 \n
8 \n
9 \n
10 \n


In [3]:
#多线程的应用例如在100万个数中找到最大的100个数， MapReduce的做法
#问题：多线程的值如何返回：用全局变量
import threading

def top3(data):
    data.sort()
    temp_result[threading.current_thread().name] = data[-3:]#data中最大的三个值
    
data_set = [
    [1,7,8,9,20,11,14,15],
    [19,21,23,24,45,12,45,56,31],
    [18,28,64,22,17,28]
]
temp_result = {} #用全局变量存储线程返回值
threads = []

for i in range(len(data_set)):#data_set有几组数据就跑几个线程
    t = threading.Thread(target = top3, name = str(i), args = (data_set[i], ))
    threads.append(t)
for t in threads:
    t.start()
for t in threads:
    t.join()

print(temp_result)
result = []
for k,v in temp_result.items():
    result.extend(v)
result.sort()
print(result[-3:])

{'0': [14, 15, 20], '1': [45, 45, 56], '2': [28, 28, 64]}
[45, 56, 64]


In [5]:
#多线程变量共享
#多线程进行数据交换时，不能够保证数据的安全性和一致性-> 提出了队列queue（自带锁）
#注意：在cmd中运行
from multiprocessing import Process, Queue
import os, time, random

def write(q):#写进程
    print("write: %s "% os.getpid()) 
    for value in ['AAA', 'BBB', 'Hello world']:
        print("write %s " %value)
        q.put(value)
        time.sleep(random.random())
        
def read(q):#读进程
    print("read: %s "% os.getpid())
    while True:
        value = q.get(True)
        print("read %s " %value)

if __name__ == "__main__" :
    q = Queue()
    pw = Process(target = write, args = (q,))
    pr = Process(target = read, args = (q,))
    pw.start()
    pr.start()
    pw.join()
    time.sleep(3)
#     pr.join() #等待read结束是没有用的，因为里面是死循环
    pr.terminate()
    print('Done')

Done


In [2]:
#锁
import threading
import time

lock = threading.Lock()
balance = 0

def change_balance(n):
    global balance
    balance +=n
    balance -=n
    print("change_balance 中的balance：", balance)
    #思考：如果是两个进程，一个加10 ，另一个加20，同时操作，最后balance的值可能是110，也可能是120
    
def run_thread(n):
    #先获取锁
    lock.acquire()
    try:#如果不检查异常，一旦有人raise Excption 这个lock就不会被释放就成了死锁（内存泄漏）
        change_balance(n)
    except:
        pass
    finally:
        lock.release()
        
threads = []
for i in range(11):
    t = threading.Thread(target =run_thread, args=(i, ))
    threads.append(t)
    
for t in threads:
    t.start()
    
for t in threads:
    t.join()

print(balance)

change_balance 中的balance： 0
change_balance 中的balance：change_balance 中的balance：  0change_balance 中的balance：0
 
0
change_balance 中的balance： 0change_balance 中的balance：
 change_balance 中的balance：change_balance 中的balance：change_balance 中的balance：change_balance 中的balance： 0   change_balance 中的balance：00
000 




0


In [4]:
#sys
import sys
print(sys.path)
print(sys.argv)

['', 'C:\\Users\\neustu\\Anaconda3\\python36.zip', 'C:\\Users\\neustu\\Anaconda3\\DLLs', 'C:\\Users\\neustu\\Anaconda3\\lib', 'C:\\Users\\neustu\\Anaconda3', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\neustu\\.ipython']
['C:\\Users\\neustu\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\neustu\\AppData\\Roaming\\jupyter\\runtime\\kernel-899f059f-5690-4fcd-873e-88f4422d17c1.json']


In [7]:
#os 应用：递归打印当前目录
import os

def dir_s(path, tabs = 0): #用户输入的是相对路径或者绝对路径
    path = os.path.abspath(path)
    files = os.listdir(path)#得到文件夹下的所有文件名称
    my_dirs = []
    for f in files:
        abs_path = os.path.join(path, f)#获取绝对路径
        if os.path.isdir(abs_path):
            my_dirs.append(f)#如果是目录则添加保存
        else :
            print('\t' * tabs + f)#如果是文件就换行打印
     #统一打印出目录的子目录
    for my_dir in my_dirs:
        print("\t" * tabs + my_dir)
        dir_s(os.path.join(path, my_dir), tabs+1)

dir_s('.')       

config.json
lesson5文件和进程.ipynb
new_sample.txt
sales.csv
sample.txt
test.py
.ipynb_checkpoints
	file-checkpoint.ipynb
	lesson5文件和进程-checkpoint.ipynb
	文件和多线程-checkpoint.ipynb
	文件和多线程课后作业-checkpoint.ipynb
1
	1.1
		1.1.1
homework5
	new_persistence.txt
	persistence.txt
	words0.txt


作业
1. 读取或制作文本文件，将全部内容倒序后写入新文件。
2. 使用线程池实现对50个文本文件进行单词出现频率统计并汇总结果

In [8]:
# 题目1

In [3]:
import os
def reverse_file(file, newFile):
    if not os.path.exists(file):
        pass
    else:
        if os.path.exists(newFile):
            os.remove(newFile)
        lines = []
        with open(file, 'r') as f:
            line = f.readline().rstrip()[::-1]
            while line :
                lines.insert(0, line)
                print('lines:', lines)
                line = f.readline().rstrip()[::-1]
                print("line:", line)
        with open(newFile, "w") as newF:
            for line in lines:
                newF.write(line + "\n")
                
abs_path = os.path.abspath('./t005/persistence.txt')    
new_abs_path = os.path.abspath('./t005/new_persistence.txt')   

if os.path.exists(abs_path):
    os.remove(abs_path)
with open(abs_path, "w") as f:
    f.write("Nothing in the world can take the place of Persistence. \n")
    f.write("Talent will not; nothing is more common than unsuccessful men with talent. Genius will not; \n")
    f.write("unrewarded genius is almost a proverb. Education will not;\n")
    f.write("the world is full of educated derelicts.\n")
    f.write("Persistence and determination alone are omnipotent. \n")
    f.write("The slogan 'Press On' has solved and always will solve the problems of the human race.\n")
    
if os.path.exists(new_abs_path):
    os.remove(new_abs_path)
reverse_file(abs_path, new_abs_path)

lines: ['.ecnetsisreP fo ecalp eht ekat nac dlrow eht ni gnihtoN']
line: ;ton lliw suineG .tnelat htiw nem lufsseccusnu naht nommoc erom si gnihton ;ton lliw tnelaT
lines: [';ton lliw suineG .tnelat htiw nem lufsseccusnu naht nommoc erom si gnihton ;ton lliw tnelaT', '.ecnetsisreP fo ecalp eht ekat nac dlrow eht ni gnihtoN']
line: ;ton lliw noitacudE .brevorp a tsomla si suineg dedrawernu
lines: [';ton lliw noitacudE .brevorp a tsomla si suineg dedrawernu', ';ton lliw suineG .tnelat htiw nem lufsseccusnu naht nommoc erom si gnihton ;ton lliw tnelaT', '.ecnetsisreP fo ecalp eht ekat nac dlrow eht ni gnihtoN']
line: .stcilered detacude fo lluf si dlrow eht
lines: ['.stcilered detacude fo lluf si dlrow eht', ';ton lliw noitacudE .brevorp a tsomla si suineg dedrawernu', ';ton lliw suineG .tnelat htiw nem lufsseccusnu naht nommoc erom si gnihton ;ton lliw tnelaT', '.ecnetsisreP fo ecalp eht ekat nac dlrow eht ni gnihtoN']
line: .tnetopinmo era enola noitanimreted dna ecnetsisreP
lines: ['.t